# Machine Learning Study Group

## Model Validation

<img src="xkcd_ML.png" style="width: 250px;">

The following comes from the sklearn examples of how to perform cross-validation on a model.

http://scikit-learn.org/stable/modules/cross_validation.html

In [1]:
import numpy as np
import scipy
from scipy.stats import randint as sp_randint

from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

### Training/testing split
sklearn has a built-in training/testing split function

In [2]:
# Load the Iris dataset
iris = datasets.load_iris()
print('Features, target shape:', iris.data.shape, iris.target.shape)

Features, target shape: (150, 4) (150,)


In [3]:
# Split with 40% of data in testing set
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.4, random_state=0)

print('Training shape (X, y):', X_train.shape, y_train.shape)
print('Testing shape (X, y):', X_test.shape, y_test.shape)

Training shape (X, y): (90, 4) (90,)
Testing shape (X, y): (60, 4) (60,)


In [4]:
# Train support vector classification
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
print('SVC score:', clf.score(X_test, y_test))

SVC score: 0.966666666667


### Cross-validation
For the iris dataset, perform support vector classification. Score the results 5 times with different splits each time.  If the estimator is a classifier and y is either binary or multiclass, StratifiedKFold is used. In all other cases, KFold is used. StratifiedKFold preserves the percentage of samples from each class.

In [5]:
def CV_model(X, y, cv, clf):
    scores = cross_val_score(clf, X, y, cv=cv)
    print('SVC cross-validation scores:', scores)
    # 95% confidence interval, mean and 2 * standard deviation
    print("Accuracy: %0.2f (+/- %0.2f)" % 
          (scores.mean(), scores.std() * 2))
    return scores

In [6]:
clf = svm.SVC(kernel='linear', C=1)
X, y = iris.data, iris.target
cv = 5    # Number of splits

scores = CV_model(X, y, cv, clf)

SVC cross-validation scores: [ 0.96666667  1.          0.96666667  0.96666667  1.        ]
Accuracy: 0.98 (+/- 0.03)


In [7]:
# Leave one out
loo = LeaveOneOut()
X = [1, 2, 3, 4]

for train, test in loo.split(X):
    print("%s %s" % (train, test))

[1 2 3] [0]
[0 2 3] [1]
[0 1 3] [2]
[0 1 2] [3]


## Hyperparameter Search

http://scikit-learn.org/stable/modules/grid_search.html

### Exhaustive Grid Search
Iterate through every possible combination of hyperparameters.

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV

In [11]:
iris = datasets.load_iris()

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10, 100, 1000]}
svr = svm.SVC()
clf = GridSearchCV(svr, parameters)
clf.fit(iris.data, iris.target)

# Print the best parameters
print(clf.cv_results_['params'][clf.best_index_])

{'C': 1, 'kernel': 'linear'}


### Randomized Search

Not all parameter values are used. A fixed number of parameter settings is sampled from the specified distributions.

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV

In [9]:
# get some data
digits = load_digits()
X, y = digits.data, digits.target

clf = RandomForestClassifier(n_estimators=20)

param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(2, 11),
              "min_samples_leaf": sp_randint(2, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

random_search.fit(X, y)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=20, n_jobs=1,
          param_distributions={'max_depth': [3, None], 'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10b5a6128>, 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10b878828>, 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10b8789b0>, 'bootstrap': [True, False], 'criterion': ['gini', 'entropy']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
   

In [10]:
# Print the best parameters
random_search.best_params_

{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 6,
 'min_samples_leaf': 5,
 'min_samples_split': 9}